# ADK 101 in 30min


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Pili Hu](https://github.com/hupili)|
| Reviewers(s) | Nil |
| Last updated | 2025-05-15 |

# Environment


In [1]:
!pip3 install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72

In [19]:

import os

API_KEY = '' # @param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
os.environ["GOOGLE_API_KEY"] = API_KEY

# PROJECT_ID = "GCP ID"  # @param {type:"string"}
# if not PROJECT_ID:
#     PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

# LOCATION = "us-central1" # @param {type:"string"}

# os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
# os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API
# [your-project-id]

In [20]:
# from google.colab import auth
# auth.authenticate_user()

# Utils

In [21]:

import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()
    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()
        agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

        if is_final_response:
            print('>>> inside final response...')
            final_response = event.content.parts[0].text
            end_time = time.time()
            elapsed_time_ms = round((end_time - start_time) * 1000, 3)
            print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-----------------------------")
        elif function_calls:
            print('+++ inside the function call...')
            for function_call in function_calls:
                print(f"function, [args]:  {function_call.name}, {function_call.args}")
        elif function_responses:
            print('--- inside the function call response...')
            # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
            # if not event.actions.pending:
            if True:
                for function_response in function_responses:
                    details = function_response.response
                    recommended_list = list(details.values())
                    print(f"Function Name: {function_response.name}")
                    result=json.dumps(recommended_list)
                    print(f"Function Results {result}")
            else:
                print(agent_res)
    print(f"Total elapsed time: {elapsed_time_ms}")


In [22]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='google.generativeai.types.content_types') # Suppress harmless warning

In [23]:
from google.adk.agents import Agent
from google.genai import types
from google.adk.tools.agent_tool import AgentTool

In [24]:
MODEL = "gemini-2.0-flash-001"

# Hello world!

Concepts:

- Agent - Starts as a wrapper of LLM that can decide actions and interact with external world (other agents, tools)
- Session - Maintains the state of one invocation. (chat log / events/ etc)
- Runner - The orchestration layer of agents. The "state machine". Runner drives agents to move the states of a session forward.

Use case:
- An agent that says hello word in a random language

In [25]:
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

In [26]:
# Constant
APP_NAME = "hello_world_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_word_agent"
MODEL = "gemini-2.0-flash-001"

# Agent
hello_world_agent = Agent(
    model=MODEL,
    name="hello_world_agent",
    description="An agent that says 'hello world'",
    instruction="""You always say 'hello world' to the user, and nothing else.
    Output 'hello world' in a random language.
    Put the language in brackets.

    Example Output 1:
    hello world (English)

    Example Output 2:
    你好，世界 (Chinese)
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_world_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)
  return events
  # for event in events:
  #     if event.is_final_response():
  #         final_response = event.content.parts[0].text
  #         print("Agent Response: ", final_response)

events = call_agent(runner, "hello")
pprint_events(events)

>>> inside final response...
Final Response (569.18 ms):
Bonjour le monde (French)

-----------------------------
Total elapsed time: 569.18


In [27]:
events = call_agent(runner, "hello")
pprint_events(events)

>>> inside final response...
Final Response (405.967 ms):
こんにちは世界 (Japanese)

-----------------------------
Total elapsed time: 405.967


In [28]:
session

Session(id='session12345', app_name='hello_world_example', user_id='user12345', state={}, events=[], last_update_time=1747282291.718386)

In [29]:
session.id

'session12345'

In [30]:
session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID)

ListSessionsResponse(sessions=[Session(id='session12345', app_name='hello_world_example', user_id='user12345', state={}, events=[], last_update_time=1747282292.299445)])

In [31]:
from pprint import pprint

pprint(session_service.sessions[APP_NAME][USER_ID][SESSION_ID].events)

[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-2fbc3f3b-e1e1-4f49-8889-6ec8abdb6c56', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='sOhfea0n', timestamp=1747282291.71987),
 Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='Bonjour le monde (French)\n')], role='model'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error

In [32]:
events = session_service.sessions[APP_NAME][USER_ID][SESSION_ID].events

In [33]:
for event in events:
  print(f"Role: {event.content.role}, Text: {event.content.parts[0].text if event.content and event.content.parts else 'N/A'}")

Role: user, Text: hello
Role: model, Text: Bonjour le monde (French)

Role: user, Text: hello
Role: model, Text: こんにちは世界 (Japanese)



# Multi turn conversation

Concepts:

- Event(s)
- Session
- SessionService: sessions[app][user][session]

Use case:

- An agent that tries to know your name and send hello USERNAME

In [34]:
# Constant
APP_NAME = "hello_name_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "hello_name_agent"
MODEL = "gemini-2.0-flash-001"

# Agent
hello_name_agent = Agent(
    model=MODEL,
    name="hello_name_agent",
    description="An agent that says 'hello USERNAME'",
    instruction="""
    You need to first ask the user's name.
    Try best to convince the user to give you a name, let it be first name, last name, or nick name.

    Once you get the user's name, say 'hello USERNAME'.
    """,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=100,
    ),
)

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=hello_name_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, session, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
  return events

events = call_agent(runner, session, "hello")
pprint_events(events)

>>> inside final response...
Final Response (800.802 ms):
Hello! Before I say hello back, could you tell me your name? It can be your first name, last name, or even a nickname! I'd love to know what to call you. 😊

-----------------------------
Total elapsed time: 800.802


In [35]:
events = call_agent(runner, session, "I don't tell you my name")
pprint_events(events)

>>> inside final response...
Final Response (782.597 ms):
I understand you might be hesitant. How about a nickname then? Just something I can use to address you! It would make our conversation a little more personal. 😊

-----------------------------
Total elapsed time: 782.597


In [36]:
events = call_agent(runner, session, "what is your name?")
pprint_events(events)

>>> inside final response...
Final Response (912.397 ms):
You can call me hello_name_agent. But I'd still really like to know *your* name! It doesn't have to be anything official, just something you'd like me to call you. Pretty please?

-----------------------------
Total elapsed time: 912.397


In [37]:
events = call_agent(runner, session, "my name is Pili")
pprint_events(events)

>>> inside final response...
Final Response (733.536 ms):
Hello Pili! It's nice to meet you. 😊

-----------------------------
Total elapsed time: 733.536


In [38]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> inside final response...
Final Response (549.215 ms):
Your name is Pili! 😊

-----------------------------
Total elapsed time: 549.215


## Switch session

In [39]:
new_session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID + '-new')

In [40]:
events = call_agent(runner, new_session, "what is my name?")
pprint_events(events)

>>> inside final response...
Final Response (753.411 ms):
I don't have access to that information. Could you tell me your name, even a first name or nickname would be great? I'd love to say hello properly!

-----------------------------
Total elapsed time: 753.411


In [41]:
events = call_agent(runner, session, "what is my name?")
pprint_events(events)

>>> inside final response...
Final Response (621.343 ms):
You told me your name is Pili! Am I forgetting something? 😊

-----------------------------
Total elapsed time: 621.343


# Use Tool

Concept:
- Tool (Python function) -- the experience of tool calling by agent (by LLM) or by human (from Python) are the same. Easy to test.

Use case:
- Simple math

In [42]:
def add(numbers: list[int]) -> int:
  """Calculates the sum of a list of integers.

    This function takes a list of integers as input and returns the sum of all
    the elements in the list.  It uses the built-in `sum()` function for
    efficiency.

    Args:
        numbers: A list of integers to be added.

    Returns:
        The sum of the integers in the input list.  Returns 0 if the input
        list is empty.

    Examples:
        add([1, 2, 3]) == 6
        add([-1, 0, 1]) == 0
        add([]) == 0
    """
  return sum(numbers)

def subtract(numbers: list[int]) -> int:
    """Subtracts numbers in a list sequentially from left to right.

    This function performs subtraction on a list of integers, applying the
    subtraction operation from left to right.  For example, given the list
    [10, 2, 5], the function will calculate 10 - 2 - 5.

    Args:
        numbers: A list of integers to be subtracted.

    Returns:
        The result of the sequential subtraction as an integer. Returns 0 if the input list is empty.

    Examples:
        subtract([10, 2, 5]) == 3  # (10 - 2) - 5 = 8 - 5 = 3
        subtract([10, 2]) == 8      # 10 - 2 = 8
        subtract([]) == 0
    """
    if not numbers:
        return 0  # Handle empty list
    result = numbers[0]
    for num in numbers[1:]:
        result -= num
    return result

def multiply(numbers: list[int]) -> int:
  """Calculates the product of a list of integers.

    This function takes a list of integers as input and returns the product of all
    the elements in the list. It iterates through the list, multiplying each
    number with the accumulated product.

    Args:
        numbers: A list of integers to be multiplied.

    Returns:
        The product of the integers in the input list. Returns 1 if the input
        list is empty.

    Examples:
        multiply([2, 3, 4]) == 24  # 2 * 3 * 4 = 24
        multiply([1, -2, 5]) == -10 # 1 * -2 * 5 = -10
        multiply([]) == 1
    """
  product = 1
  for num in numbers:
    product *= num
  return product

def divide(numbers: list[int]) -> float:  # Use float for division
    """Divides numbers in a list sequentially from left to right.

    This function performs division on a list of integers, applying the division
    operation from left to right.  For example, given the list [10, 2, 5], the
    function will calculate 10 / 2 / 5.

    Args:
        numbers: A list of integers to be divided.

    Returns:
        The result of the sequential division as a float.

    Raises:
        ZeroDivisionError: If any number in the list *after* the first element
                           is zero, a ZeroDivisionError is raised.  Division by
                           zero is not permitted.

    Returns:
        float: The result of the division. Returns 0.0 if the input list is empty.

    Examples:
        divide([10, 2, 5]) == 1.0  # (10 / 2) / 5 = 5 / 5 = 1.0
        divide([10, 2]) == 5.0      # 10 / 2 = 5.0
        divide([10, 0, 5])  # Raises ZeroDivisionError
        divide([]) == 0.0
    """
    if not numbers:
        return 0.0 # Handle empty list
    if 0 in numbers[1:]: # Check for division by zero
        raise ZeroDivisionError("Cannot divide by zero.")
    result = numbers[0]
    for num in numbers[1:]:
        result /= num
    return result


In [43]:
add([1, 2, 3])

6

In [44]:
multiply([2, 2, 3])

12

In [45]:
simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

In [46]:
import random

def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  session_service = InMemorySessionService()
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service)
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call



In [47]:
call = caller_factory(root_agent=simple_math_agent)

In [48]:
pprint_events(call('hello'))

>>> inside final response...
Final Response (1006.415 ms):
Hello! I am ready to perform calculations. What would you like to do? For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'. You can also provide a range: 'Multiply the numbers between 1 and 10'.

-----------------------------
Total elapsed time: 1006.415


In [51]:
pprint_events(call('what is three plus 9?'))

+++ inside the function call...
function, [args]:  add, {'numbers': [3, 9]}
--- inside the function call response...
Function Name: add
Function Results [12]
>>> inside final response...
Final Response (1276.106 ms):
3 + 9 = 12

-----------------------------
Total elapsed time: 1276.106


In [52]:
pprint_events(call('multiply that by 2'))

+++ inside the function call...
function, [args]:  multiply, {'numbers': [12, 2]}
--- inside the function call response...
Function Name: multiply
Function Results [24]
>>> inside final response...
Final Response (1255.846 ms):
12 multiplied by 2 is 24.

-----------------------------
Total elapsed time: 1255.846


In [53]:
call = caller_factory(root_agent=simple_math_agent)
pprint_events(call('Here is my math problem that is about apple counting. Let us start saying that I have three apple'))

>>> inside final response...
Final Response (710.327 ms):
Okay. I am ready to perform calculations on numbers. What do you want to do with the number three? For example, do you want to add more apples, subtract some apples, multiply by a number, or divide?

-----------------------------
Total elapsed time: 710.327


In [54]:
pprint_events(call('Alice gave anoter 2 apples'))

>>> inside final response...
Final Response (696.892 ms):
Okay, so you want to add 2 apples to your existing 3 apples. Is that right?

-----------------------------
Total elapsed time: 696.892


In [55]:
pprint_events(call('how many apples do I have?'))

+++ inside the function call...
function, [args]:  add, {'numbers': [3, 2]}
--- inside the function call response...
Function Name: add
Function Results [5]
>>> inside final response...
Final Response (1240.136 ms):
You have 5 apples.

-----------------------------
Total elapsed time: 1240.136


In [56]:
pprint_events(call('Bob gave me 3 apples. how many apples do I have?'))

+++ inside the function call...
function, [args]:  add, {'numbers': [5, 3]}
--- inside the function call response...
Function Name: add
Function Results [8]
>>> inside final response...
Final Response (1391.161 ms):
Now you have 8 apples.

-----------------------------
Total elapsed time: 1391.161


In [57]:
pprint_events(call('Alice and Bob do this to me for three more times. how many apples do I have?'))

+++ inside the function call...
function, [args]:  add, {'numbers': [8, 15]}
--- inside the function call response...
Function Name: add
Function Results [23]
>>> inside final response...
Final Response (1991.234 ms):
So you will have 23 apples in total.

-----------------------------
Total elapsed time: 1991.234


**NOTE: The above calculation is wrong**

# Agent As Tool

Tool: If the execution flow goes from Agent to Tool and it always come back to the initiation agent.

Agent: If the execution flow goes from Agent to **another** agent, the latter agent can decide where the flow goes next.

Concepts:
- AgentTool -- If you want another agent to handle a task and always come back to the caller, make this `Agent` a `Tool` by `AgentTool`

Use case:
- Advanced math agent

In [59]:
agent_math_advanced_instruction = '''
I am an advanced math agent. I handle user query in the below steps:

1. I shall analyse the chat log to understand current question and make a math formula for it.
2. Break down a complex compuation based on arithmetic priority and hand over to simple_math_agent for the calculation.
3. Note that simple_math_agent can only understand numbers, so I need to convert natural language expression of numbers into digits.

<example>
<input> alice gives us 3 apples, bob gives us 5 apples. They do this seven times. Then we eat four apples. How many apples do we have now? </input>
<think> what is (3+5) * 7 -4 </think>
<think>I need to first calculate (3+5) as the highest priority operation.</think>
<call_tool> pass (3+5) to simple_math_agent </call_tool>
<tool_response>8</tool_response>
<think> The question now becomes 8 * 7 - 4, and next highest operation is 8 * 7</think>
<call_tool> pass 8 * 7 to simple_math_agent </call_tool>
<tool_response>56</tool_response>
<think> The question now becomes 56 - 4, and next highest operation is 56 - 4</think>
<call_tool> pass 56 - 4 to simple_math_agent </call_tool>
<tool_response>52</tool_response>
<think>There is a single number, so it is the final answer.</think>
<output>The result of "(3+5) * 7 - 4" is 52</output>
</example>
'''

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [60]:
call = caller_factory(root_agent=agent_math_advanced)

In [62]:
pprint_events(call('who are you?'))

>>> inside final response...
Final Response (719.737 ms):
I am an advanced math agent that can break down a complex computation into multiple simple operations and use a simple math agent to solve them.

-----------------------------
Total elapsed time: 719.737


In [64]:
pprint_events(call('what is 1 + (three+2) times 7 '))

+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '3+2'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The result is 5.\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '5*7'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["35\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '1+35'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["36\n"]
>>> inside final response...
Final Response (7281.381 ms):
The result of "1 + (3+2) times 7" is 36.

-----------------------------
Total elapsed time: 7281.381


In [65]:
call = caller_factory(root_agent=agent_math_advanced)

In [66]:
pprint_events(call('add from 1 to 2. then multiply that by 2.'))


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '1+2'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The answer is 3.\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '3*2'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The result is 6.\n"]
>>> inside final response...
Final Response (4756.467 ms):
The result of "add from 1 to 2. then multiply that by 2" is 6.

-----------------------------
Total elapsed time: 4756.467


In [67]:
call = caller_factory(root_agent=agent_math_advanced)
pprint_events(call('how much is 10 / 5 + three times 5?'))

+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '10 / 5'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The answer is 2.0\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '3 * 5'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["15\n"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2.0 + 15'}
--- inside the function call response...
Function Name: simple_math_agent
Function Results ["I am designed to work with integers. Could you provide me with integers only?"]


+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2 + 15'}


--- inside the function call response...
Function Name: simple_math_agent
Function Results ["The sum of 2 and 15 is 17.\n"]
>>> inside final response...
Final Response (8350.533 ms):
The result of "10 / 5 + three times 5" is 17

-----------------------------
Total elapsed time: 8350.533


**NOTE**: This part is very nice, it shows how two agents interact to clarify the input type should be integers and input again.

```
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2.0 + 15'}
--- inside the function call response...
Function Name: simple_math_agent
Function Results ["I can only work with integers. Could you provide the numbers as integers?\n"]
WARNING:google_genai.types:Warning: there are non-text parts in the response: ['function_call'],returning concatenated text from text parts,check out the non text parts for full response from model.
+++ inside the function call...
function, [args]:  simple_math_agent, {'request': '2 + 15'}

```

# Output Format Control

In [68]:
from typing import List
from pydantic import BaseModel, Field
from google.genai import types

json_response_config = types.GenerateContentConfig(
  response_mime_type="application/json",
)

class OutputSchema(BaseModel):
    original_query: str = Field(description="The original text from user.")
    corrected_text: str = Field(description="The corrected text.")
    errors: List[str] = Field(description="An array of descriptions of each error.")
    explanations: List[str] = Field(description="An array of explanations for each correction.")

json_schema = OutputSchema.model_json_schema()
json_schema

{'properties': {'original_query': {'description': 'The original text from user.',
   'title': 'Original Query',
   'type': 'string'},
  'corrected_text': {'description': 'The corrected text.',
   'title': 'Corrected Text',
   'type': 'string'},
  'errors': {'description': 'An array of descriptions of each error.',
   'items': {'type': 'string'},
   'title': 'Errors',
   'type': 'array'},
  'explanations': {'description': 'An array of explanations for each correction.',
   'items': {'type': 'string'},
   'title': 'Explanations',
   'type': 'array'}},
 'required': ['original_query', 'corrected_text', 'errors', 'explanations'],
 'title': 'OutputSchema',
 'type': 'object'}

In [69]:
# 1. The {json_schema} in instruction is the key for model to follow the schema.
# 2. The output_schema=OutputSchema provides a validation step after model output.

agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
)

In [70]:
call = caller_factory(root_agent=agent_grammar, app_name='grammar', user_id='User12345', session_id='333')

In [71]:
session_service.sessions.keys()#['grammar']['User12345']['333']

dict_keys(['hello_name_example'])

In [72]:
pprint_events(call('ho much is three times 5?'))

>>> inside final response...
Final Response (842.087 ms):
{
  "original_query": "ho much is three times 5?",
  "corrected_text": "How much is three times 5?",
  "errors": [
    "Typo"
  ],
  "explanations": [
    "There was a typo. 'ho' should be 'How'."
  ]
}
-----------------------------
Total elapsed time: 842.087


In [73]:
pprint_events(call('whut will be 2 plus 3?'))

>>> inside final response...
Final Response (1341.039 ms):
{
  "original_query": "whut will be 2 plus 3?",
  "corrected_text": "What will be 2 plus 3?",
  "errors": [
    "Misspelling",
    "Incorrect word choice"
  ],
  "explanations": [
    "'Whut' is not the correct spelling of 'What.'",
    "The word 'be' is unnecessary in the sentence. It should be removed."
  ]
}
-----------------------------
Total elapsed time: 1341.039


In [74]:
# session_service.sessions['grammar']['User12345']['333']

# Child Agent Sequential

Concept:
- Flow: sequential

Use case:
- Teaching Assistant

In [76]:
agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
)

simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    # children=[agent_math],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [77]:
from google.adk.agents.sequential_agent import SequentialAgent

agent_teaching_assistant_sequential = SequentialAgent(
    name="agent_teaching_assistant_sequential",
    description="This agent acts as a friendly teaching assistant, checking the grammar of kids' questions, performing math calculations using corrected or original text (if grammatically correct), and providing results or grammar feedback in a friendly tone.",
    sub_agents=[agent_grammar, simple_math_agent],
)

In [78]:
call = caller_factory(root_agent=agent_teaching_assistant_sequential)

In [79]:
pprint_events(call('what is 1+2?'))

>>> inside final response...
Final Response (1525.388 ms):
{
  "original_query": "what is 1+2?",
  "corrected_text": "What is 1 + 2?",
  "errors": [
    "Missing capitalization at the beginning of the sentence.",
    "Missing spaces around the '+' operator.",
    "Missing question mark at the end of the sentence."
  ],
  "explanations": [
    "Sentences always start with a capital letter!",
    "When you write math problems, it's easier to read if you put spaces around the + sign.",
    "We put a question mark at the end of a sentence when we are asking something."
  ]
}
-----------------------------


+++ inside the function call...
function, [args]:  add, {'numbers': [1, 2]}
--- inside the function call response...
Function Name: add
Function Results [3]
>>> inside final response...
Final Response (2756.084 ms):
The answer is 3.

-----------------------------
Total elapsed time: 2756.084


In [80]:
pprint_events(call('ho much  is three * 2?'))

>>> inside final response...
Final Response (1238.429 ms):
{
  "original_query": "ho much  is three * 2?",
  "corrected_text": "How much is three times two?",
  "errors": [
    "Misspelling",
    "Missing word",
    "Incorrect symbol usage"
  ],
  "explanations": [
    "The word 'how' was spelled as 'ho'.",
    "The word 'much' was missing.",
    "The symbol * represents multiplication, but it is better to write 'times' for kids."
  ]
}
-----------------------------


+++ inside the function call...
function, [args]:  multiply, {'numbers': [3, 2]}
--- inside the function call response...
Function Name: multiply
Function Results [6]
>>> inside final response...
Final Response (2580.135 ms):
It is 6.

-----------------------------
Total elapsed time: 2580.135


# Child Agent Loop

Concepts:
- Flow: Loop
- Event.action.escalate

Use case:
- Debate agent?

In [101]:
agent_grammar = Agent(
    model=MODEL,
    name='agent_grammar',
    description="This agent corrects grammar mistakes in text provided by children, explains the errors in simple terms, and returns both the corrected text and the explanations.",
    instruction=f"""
        You are a friendly grammar helper for kids.  Analyze the following text,
        correct any grammar mistakes, and explain the errors in a way that a
        child can easily understand.  Don't just list the errors; explain them
        in a paragraph using simple but concise language.

        Output in a JSON object with the below schema:
        {json_schema}
    """,
    output_schema=OutputSchema,
    generate_content_config=json_response_config,
)

simple_math_agent = Agent(
    model=MODEL,
    name="simple_math_agent",
    description="This agent performs basic arithmetic operations (addition, subtraction, multiplication, and division) on user-provided numbers, including ranges.",
    instruction="""
      I can perform addition, subtraction, multiplication, and division operations on numbers you provide.
      Tell me the numbers you want to operate on.
      For example, you can say 'add 3 5', 'multiply 2, 4 and 3', 'Subtract 10 from 20', 'Divide 10 by 2'.
      You can also provide a range: 'Multiply the numbers between 1 and 10'.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[add, subtract, multiply, divide],
)

agent_math_advanced = Agent(
    model=MODEL,
    name="agent_math_advanced",
    description="The advanced math agent can break down a complex computation into multiple simple operations and use math_agent to solve them.",
    instruction=agent_math_advanced_instruction,
    tools=[AgentTool(agent=simple_math_agent)],
    # children=[agent_math],
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
)

In [102]:
from google.adk.tools.tool_context import ToolContext

def exit_loop(tool_context: ToolContext):
  tool_context.actions.escalate = True

agent_checker = Agent(
    model=MODEL,
    name="agent_checker",
    description="This agent checks if the kid's query is fulfilled. It escalates if both grammar is fixed and the math is calculated",
    instruction="""
    Analyze the chat log between the user (kids) and other agents.

    We are expecting two outcomes from the conversation.
    1. agent_grammar helps to fix the grammar of kid's question.
    2. agent_math_advanced tries to solve the math step by step.

    Decide the action and respond as follows:

    1. If you find user question is incomplete in the chat log, ask a clarification question to the kid. Then call `exit_loop`.

    2. If both agent_grammar and agent_math_advanced have done their task, summarize the answer in a friendly way to the kid. Then call `exit_loop`.

    3. If there is no pending math problem to solve, friendly tell the kid you help fix grammar and solve math, but does not know anything else. Politely ask for math question. Then call exit_loop.

    4. If agent_grammar has fixed the grammar but agent_math_advanced has not generated a final answer with single number, extract corrected_text from most recent JSON response and empahsize that is the math question to solve.

    5. Otherwise, say "Thanks for the question! I'll let grammar and math agent to help you!". Do not call any tools in this case.

    """,
    tools=[exit_loop]
)

agent_extractor = Agent(
    model=MODEL,
    name="agent_extractor",
    description="Extract corrected_text from most recent JSON response and empahsize that is the math question to solve.",
    instruction="""
    Extract corrected_text from most recent JSON response and empahsize that is the math question to solve.
    """,
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    input_schema=OutputSchema,
)


In [103]:
from google.adk.agents.loop_agent import LoopAgent

agent_teaching_assistant_loop = LoopAgent(
    name="agent_teaching_assistant_loop",
    description="This agent acts as a friendly teaching assistant, checking the grammar of kids' questions, performing math calculations using corrected or original text (if grammatically correct), and providing results or grammar feedback in a friendly tone.",
    sub_agents=[agent_checker, agent_grammar, agent_extractor, agent_math_advanced],
)

In [104]:
call = caller_factory(root_agent=agent_teaching_assistant_loop)

In [105]:
pprint_events(call('ho much  is three * 2?'))

>>> inside final response...
Final Response (554.277 ms):
Thanks for the question! I'll let grammar and math agent to help you!

-----------------------------
>>> inside final response...
Final Response (1712.86 ms):
{
  "original_query": "ho much  is three * 2?",
  "corrected_text": "How much is three times two?",
  "errors": [
    "Misspelling",
    "Incorrect word choice"
  ],
  "explanations": [
    "\"Ho\" should be spelled \"How.\"",
    "When asking about multiplication, we say \"times\" instead of \"*\"."
  ]
}
-----------------------------
>>> inside final response...
Final Response (2199.843 ms):
The math question to solve is: **How much is three times two?**

-----------------------------
>>> inside final response...
Final Response (2950.532 ms):
<think> The question is three times two, which is 3 * 2 </think>
<call_tool> pass 3 * 2 to simple_math_agent </call_tool>

-----------------------------
>>> inside final response...
Final Response (3584.304 ms):
For context:[agent_m

+++ inside the function call...
function, [args]:  exit_loop, {}
--- inside the function call response...
Function Name: exit_loop
Function Results []
Total elapsed time: 5867.992


In [106]:
pprint_events(call('whut is 1 plus 3 * 5?'))

>>> inside final response...
Final Response (524.442 ms):
Thanks for the question! I'll let grammar and math agent to help you!

-----------------------------
>>> inside final response...
Final Response (1540.556 ms):
{
  "original_query": "whut is 1 plus 3 * 5?",
  "corrected_text": "What is 1 plus 3 times 5?",
  "errors": [
    "Misspelling",
    "Incorrect word choice"
  ],
  "explanations": [
    "\"Whut\" is not a real word. The correct word is \"What.\"",
    "When writing out a math problem, we say \"times\" instead of using the * symbol."
  ]
}
-----------------------------
>>> inside final response...
Final Response (2219.633 ms):
The math question to solve is: **What is 1 plus 3 times 5?**

-----------------------------
>>> inside final response...
Final Response (2856.272 ms):
<think> The question is 1 + 3 * 5. I need to calculate 3 * 5 first. </think>
<call_tool> pass 3 * 5 to simple_math_agent </call_tool>

-----------------------------
>>> inside final response...
Final R

+++ inside the function call...
function, [args]:  exit_loop, {}
--- inside the function call response...
Function Name: exit_loop
Function Results []
Total elapsed time: 9458.766


In [114]:
#TODO: need more examination of the prompt to make the sequence work.
# call = caller_factory(root_agent=agent_teaching_assistant_loop)

In [115]:
# pprint_events(call('hello'))

In [116]:
# pprint_events(call('what is three plus another number?'))

In [117]:
# pprint_events(call('yup, the other number is 5.'))

# State

Concept:

- session.state -- passing values around.
- session_service.sessions[app_id][user_id][session_is].state
- ToolContext.state
- '{var}' notation in prompt, to access state['var']

Use case:

- Task management

In [118]:
import random
from google.adk.agents import Agent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.agents.sequential_agent import SequentialAgent
from google.adk.agents.loop_agent import LoopAgent
from google.adk.tools.tool_context import ToolContext
from typing import Tuple

MODEL = "gemini-2.0-flash-001"

def exit_loop(tool_context: ToolContext):
  tool_context.actions.escalate = True

def roll_dice() -> int:
  """Roll a dice. Return a number between 1 and 6 (inclusive).

  Returns:
      int: A random integer between 1 and 6 (inclusive).
  """
  return random.randint(1, 6)

def update_results(player1: int, player2: int, tool_context: ToolContext) -> str:
  '''Updates the current round of dice rolling results and returns the cumulative results.

  This function takes the results of a single round of dice rolling for two players,
  updates the cumulative scores for each player, and returns the updated cumulative
  scores. It also clears the previous round's results.

  Args:
      player1: The result of player 1's dice roll for the current round.
      player2: The result of player 2's dice roll for the current round.
      tool_context: The tool context object containing the current state.

  Returns:
      A string containing the updated cumulative scores for player 1 and player 2.
  '''
  state = tool_context.state
  if 'play1_total' not in state:
    state['play1_total'] = 0
  state['play1_total'] += player1
  if 'play2_total' not in state:
    state['play2_total'] = 0
  state['play2_total'] += player2
  # Clear the previous round
  state['player_1_result'] = ''
  state['player_2_result'] = ''
  return f"Cumulative points so far. player 1: {state['play1_total']}, player 2: {state['play2_total']}"

player1 = Agent(
    model=MODEL,
    name='player_1',
    description='Play 1',
    instruction='''
    Roll a dice with tool roll_dice. Tell the result in a funny and creative way.
    Do not look at previous rounds result.
    Do not look at at the other player's result.
    Do not respond to the user.
    Only roll your own dice and tell everyone the result.
    ''',
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[roll_dice],
    output_key='player_1_result',
)

player2 = Agent(
    model=MODEL,
    name='player_2',
    description='Play 2',
    instruction='''
    Roll a dice with tool roll_dice. Tell the result in a funny and creative way.
    Do not look at previous rounds result.
    Do not look at at the other player's result.
    Do not respond to the user.
    Only roll your own dice and tell everyone the result.
    ''',
    generate_content_config=types.GenerateContentConfig(temperature=0.2),
    tools=[roll_dice],
    output_key='player_2_result',
)

judge = Agent(
    model=MODEL,
    name='judge',
    description='Judge',
    instruction='''Analyze both players result and make a short summary of this round.
    Only say the numeric value of player 1 and player 2 result.

    Use the tool `update_results` to add player 1 and player 2 current round results to our scoreboard,
    and get their cumulative results returned by the tool.

    If any player's cumulative result is greater than 15, call `exit_loop` to end the game.
    Otherwise, let the game continue.

    Output in the format:
    - First state both player's current round result
    - Second state both player's cumulative result
    - Third state which player is leading

    Following is your input:

    Play 1 result: {player_1_result}
    Play 2 result: {player_2_result}
    ''',
    tools=[update_results, exit_loop],
)

dice_game = LoopAgent(
    name="dice_game",
    description="A dice game that player 1, player 2 and judge take turns to act.",
    sub_agents=[player1, player2, judge],
)

In [119]:
call = caller_factory(root_agent=dice_game, app_name='dice_game', user_id='User12345', session_id='333')

In [120]:
events = call('start')

In [121]:
state = session_service.sessions#['dice_game']['User12345']['333'].state
state.keys()

dict_keys(['hello_name_example'])

In [122]:
start_time = time.time()
for _, event in enumerate(events):
    is_final_response = event.is_final_response()
    function_calls = event.get_function_calls()
    function_responses = event.get_function_responses()
    agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

    print(f'+++state+++\n{state}\n---state---')

    if is_final_response:
        print('>>> inside final response...')
        final_response = event.content.parts[0].text
        end_time = time.time()
        elapsed_time_ms = round((end_time - start_time) * 1000, 3)
        print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
        print("-----------------------------")
    elif function_calls:
        print('+++ inside the function call...')
        for function_call in function_calls:
            print(f"function, [args]:  {function_call.name}, {function_call.args}")
    elif function_responses:
        print('--- inside the function call response...')
        # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
        # if not event.actions.pending:
        if True:
            for function_response in function_responses:
                details = function_response.response
                recommended_list = list(details.values())
                print(f"Function Name: {function_response.name}")
                result=json.dumps(recommended_list)
                print(f"Function Results {result}")
        else:
            print(agent_res)
print(f"Total elapsed time: {elapsed_time_ms}")

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

...

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

+++state+++
{'hello_name_example': {'user12345': {'session12345': Session(id='session12345', app_name='hello_name_example', user_id='user12345', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='hello')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, invocation_id='e-82b8536f-f0dc-4fb3-b87c-90ca575c3e81', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='Ppk9EPMh', timestamp=1747282297.225992), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=No

# ADK Web UI



In [123]:
!rm -rf google-adk-101
!git clone https://github.com/hupili/google-adk-101.git

Cloning into 'google-adk-101'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 15 (delta 1), reused 15 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), done.
Resolving deltas: 100% (1/1), done.


In [124]:
!ls google-adk-101/sample_agents/agent_dice

agent.py  __init__.py


In [125]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [133]:
# Run this command from the shell (lower left corner of CoLab)
# npx localtunnel --port 8000
!npx localtunnel --port 8000 &> localtunnel.log &

In [127]:
!cat localtunnel.log

your url is: https://rich-pianos-exist.loca.lt


In [134]:
# Run this in the CoLab to get the password, which is basically an IP address.
!curl https://loca.lt/mytunnelpassword

34.75.252.45

In [129]:
# Run this to launch the UI of ADK later
!cd google-adk-101/sample_agents/ && adk web

INFO:     Started server process [7181]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http://localhost:8000.                         |
+-----------------------------------------------------------------------------+

INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     61.93.123.74:0 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     61.93.123.74:0 - "GET /dev-ui HTTP/1.1" 200 OK
INFO:     61.93.123.74:0 - "GET /styles-4VDSPQ37.css HTTP/1.1" 200 OK
INFO:     61.93.123.74:0 - "GET /polyfills-FFHMD2TL.js HTTP/1.1" 200 OK
INFO:     61.93.123.74:0 - "GET /main-ULN5R5I5.js HTTP/1.1" 200 OK
INFO:     61.93.123.74:0 - "GET /assets/config/runtime-config.json HTT

## Cleanup resources

In [135]:
!ps aux | grep localtunnel

root        7930  4.3  0.4 1164796 65204 ?       Sl   04:29   0:00 npm exec localtunnel --port 8000
root        8019  0.0  0.0   7376  3584 ?        S    04:29   0:00 /bin/bash -c ps aux | grep localtunnel
root        8021  0.0  0.0   6484  2248 ?        S    04:29   0:00 grep localtunnel


In [140]:
!kill -15 7930

In [141]:
!ps aux | grep localtunnel

root        8199  0.0  0.0   7376  3440 ?        S    04:30   0:00 /bin/bash -c ps aux | grep localtunnel
root        8201  0.0  0.0   6484  2272 ?        S    04:30   0:00 grep localtunnel
